# Business Understanding

The company is looking to expand into new industries to diversify its portfolio. They are primarily interested in purchasing and operating airplanes for commercial enterprises, and are wanting to look into the potential risks of aircraft. Through the use of aviation accident data recorded by the Natiaonl Transportation Safety Board, we will identify aircraft that pose the lowest risk to help the company come to a decision for its first aircraft purchases.

# Data Understanding

The aviation accident data recorded by the National Transportation Safety Board, includes aviation accidents recorded from 1962 to 2023. This data contains records for civil aviation accidents and selected incidents in the United States as well as international waters for aerial vehicles ranging from gliders and balloons, to jet planes for both commercial and military.

In [1]:
# Importing the necessary tools
import pandas as pd
import matplotlib as plt
%matplotlib inline

pd.options.mode.copy_on_write = True

In [2]:
# Importing 'Aviation_Data.csv'
aviation_data = pd.read_csv('Aviation_Data.csv' )

<ipython-input-2-fb490852ed91>:2: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_data = pd.read_csv('Aviation_Data.csv' )


## Data Preparation/Cleaning

For Data Preperation and cleaning, I first create a new dataframe from the aviation data that contains the columns that will be the most benificial. From there I normalize the column names and start dropping rows with with values that do not fit the commercial plane criteria.

In [3]:
# Creating a new dataframe with specific columns
avi_records = aviation_data[['Aircraft.Category', 'Make', 'Model', 'Engine.Type', 'Number.of.Engines',
                             'Aircraft.damage', 'Injury.Severity', 'Weather.Condition', 'Event.Date']]
avi_records.head()

,Aircraft.Category,Make,Model,Engine.Type,Number.of.Engines,Aircraft.damage,Injury.Severity,Weather.Condition,Event.Date
0,NaN,Stinson,108-3,Reciprocating,1.0,Destroyed,Fatal(2),UNK,1948-10-24
1,NaN,Piper,PA24-180,Reciprocating,1.0,Destroyed,Fatal(4),UNK,1962-07-19
2,NaN,Cessna,172M,Reciprocating,1.0,Destroyed,Fatal(3),IMC,1974-08-30
3,NaN,Rockwell,112,Reciprocating,1.0,Destroyed,Fatal(2),IMC,1977-06-19
4,NaN,Cessna,501,NaN,NaN,Destroyed,Fatal(1),VMC,1979-08-02


In [4]:
avi_records = avi_records.rename(columns=lambda name: name.replace('.', '_'))
avi_records.head(3)

,Aircraft_Category,Make,Model,Engine_Type,Number_of_Engines,Aircraft_damage,Injury_Severity,Weather_Condition,Event_Date
0,NaN,Stinson,108-3,Reciprocating,1.0,Destroyed,Fatal(2),UNK,1948-10-24
1,NaN,Piper,PA24-180,Reciprocating,1.0,Destroyed,Fatal(4),UNK,1962-07-19
2,NaN,Cessna,172M,Reciprocating,1.0,Destroyed,Fatal(3),IMC,1974-08-30


In [5]:
# Turning strings in 'Make' to title to make values uniform
avi_records['Make'] = avi_records['Make'].str.title()
avi_records.head(3)

,Aircraft_Category,Make,Model,Engine_Type,Number_of_Engines,Aircraft_damage,Injury_Severity,Weather_Condition,Event_Date
0,NaN,Stinson,108-3,Reciprocating,1.0,Destroyed,Fatal(2),UNK,1948-10-24
1,NaN,Piper,PA24-180,Reciprocating,1.0,Destroyed,Fatal(4),UNK,1962-07-19
2,NaN,Cessna,172M,Reciprocating,1.0,Destroyed,Fatal(3),IMC,1974-08-30


In [6]:
avi_records['Event_Date'] = pd.to_datetime(avi_records['Event_Date'], format='ISO8601')

In [7]:
# Dropping rows with unwanted engine types and keeping NaN
unwanted_eng_type = ['Reciprocating', 'Turbo Prop', 'Electric', 'Hybrid Rocket', 'Turbo Shaft',
                    'LR', 'NONE', 'UNK']
avi_records = avi_records[~avi_records['Engine_Type'].isin(unwanted_eng_type)]

In [8]:
# Dropping rows with unwanted Aircraft Category and keeping NaN
unwanted_craft_cate = ['Helicopter', 'Glider', 'Balloon', 'Weight-Shift','Gyrocraft', 'WSFT', 
                       'Ultralight', 'Powered Parachute', 'Powered-Lift', 'ULTR', 'UNK', 'Unknown']
avi_records = avi_records[~avi_records['Aircraft_Category'].isin(unwanted_craft_cate)]

In [9]:
# Dropping rows with unwanted number of engines and keeping NaN
unwanted_num_engine = [0, 1, 6]
avi_records = avi_records[~avi_records['Number_of_Engines'].isin(unwanted_num_engine)]

According to an artical from [Investopedia](https://www.investopedia.com/ask/answers/050415/what-companies-are-major-players-airline-supply-business.asp 'Who Are the Major Airplane Manufacturing Companies?') The two major aircraft manufacturers for commercial use are Boeing and Airbus. I first double check if there are any company name variances in the dataset, and then replace the variances with the respective company's commonly used name.

In [10]:
# First checking to see if company names were inputed properly
avi_records[avi_records['Make'].str.contains('Boeing', case=False, na=False)].Make.value_counts()

Make
Boeing                            2344
Boeing Company                       9
The Boeing Company                   9
Boeing Vertol                        1
Boeing Company, Long Beach Div       1
Boeing 777-306Er                     1
Boeing Commercial Airplane Gro       1
Name: count, dtype: int64

In [11]:
avi_records[avi_records['Make'].str.contains('Airbus', case=False, na=False)].Make.value_counts()

Make
Airbus               270
Airbus Industrie     161
Airbus Industries      1
Name: count, dtype: int64

In [12]:
# Replacing Boeing and Airbus name variants with commonly used names.
avi_records['Make'] = avi_records['Make'].replace(['Boeing Company', 'The Boeing Company', 'Boeing Vertol',
                                                   'Boeing Company, Long Beach Div', 'Boeing 777-306Er',
                                                   'Boeing Commercial Airplane Gro'], 'Boeing')

In [13]:
avi_records['Make'] = avi_records['Make'].replace(['Airbus Industrie', 'Airbus Industries'], 'Airbus')

After cleaning the names, I create a dataframe for each individual company to further clean as well as further narrow the values in the 'Model' column to show aircraft still in production for [Airbus](https://www.airbus.com/en/products-services/commercial-aircraft/passenger-aircraft 'Airbus Passenger Aircraft'), and [Boeing](https://www.boeing.com/commercial#products-and-services 'Boeing Commercial Aircraft') based off of the individual companies website product list.

In [14]:
# Creating a dataframe with Boeing specific aircraft
boeing = avi_records[avi_records['Make'].str.contains('Boeing', na=False)]
boeing.head(3)

,Aircraft_Category,Make,Model,Engine_Type,Number_of_Engines,Aircraft_damage,Injury_Severity,Weather_Condition,Event_Date
84,NaN,Boeing,737-222,Turbo Fan,2.0,Destroyed,Fatal(78),IMC,1982-01-13
320,Airplane,Boeing,B737-2H4,Turbo Jet,2.0,Minor,Incident,IMC,1982-02-15
344,NaN,Boeing,727-235,Turbo Fan,3.0,Substantial,Non-Fatal,VMC,1982-02-17


In [15]:
boeing_production = ['B777', 'B-777', '777', '737-291', '737']
boeing = boeing[boeing['Model'].isin(boeing_production)]

After some research on the B777, and 737-291 I concluded that these planes are the same Models as the 777 and 737 aircrafts so I replace those values with their commonly used model name.

In [16]:
# Cleaning the Boeing dataframe
boeing['Model'] = boeing['Model'].str.replace('B-777', '777')

In [17]:
boeing['Model'] = boeing['Model'].str.replace('B777', '777')

In [18]:
boeing['Model'] = boeing['Model'].str.replace('737-291', '737')

In [19]:
boeing['Engine_Type'] = boeing['Engine_Type'].fillna('Turbo Fan')

In [20]:
boeing['Engine_Type'] = boeing['Engine_Type'].replace('Unknown', 'Turbo Fan')

In [21]:
boeing['Number_of_Engines'] = boeing['Number_of_Engines'].fillna(2.0)

In [22]:
boeing['Weather_Condition'] = boeing['Weather_Condition'].fillna('UNK')

In [23]:
boeing['Aircraft_Category'] = boeing['Aircraft_Category'].fillna('Airplane')

In [42]:
boeing= boeing.dropna(subset= ['Aircraft_damage', 'Injury_Severity', 'Event_Date'])

In [25]:
boeing['Injury_Severity'] = boeing['Injury_Severity'].apply(lambda x: ''.join(y for y in x if not y.isdigit()))

In [26]:
boeing['Injury_Severity'] = boeing['Injury_Severity'].str.replace('Fatal()', 'Fatal')
boeing['Injury_Severity'].value_counts()

Injury_Severity
Non-Fatal    136
Fatal         18
Incident       6
Name: count, dtype: int64

In [27]:
# Creating a dataframe with Airbus specific aircraft
airbus = avi_records[avi_records['Make'].str.contains('Airbus', na=False)]
airbus.head(3)

,Aircraft_Category,Make,Model,Engine_Type,Number_of_Engines,Aircraft_damage,Injury_Severity,Weather_Condition,Event_Date
6737,NaN,Airbus,A-300B4-2C,Turbo Fan,2.0,Minor,Incident,VMC,1983-11-06
11775,NaN,Airbus,A-300B4-2C,Turbo Fan,2.0,NaN,Non-Fatal,IMC,1985-05-28
13699,NaN,Airbus,A-300 B4-203,Turbo Fan,2.0,Minor,Incident,VMC,1985-12-30


In [28]:
# updating airbus dataframe to have only aircraft still in production
airbus_production = ['A320-211', 'A-320-211', 'A-320', 'A320', 'A330',
                     'A320-321', 'A320-233', '320', 'A321', 'Airbus A330',
                     'A320 211', '321', 'A320 - 216', 'A320 - 211', '330', 
                     'A320 233', 'A321-271N', 'A220', 'A320-271N', 'A320-251N',
                     '220']
airbus = airbus[airbus['Model'].isin(airbus_production)]

In [29]:
# Cleaning the Airbus dataframe
airbus['Model'] = airbus['Model'].str.replace('A320 233', 'A320-233')

In [30]:
airbus['Model'] = airbus['Model'].str.replace('A320 211', 'A320-211')

In [31]:
airbus['Model'] = airbus['Model'].str.replace('Airbus A330', 'A330')

In [32]:
airbus['Model'] = airbus['Model'].str.replace(' ', '')

In [33]:
airbus['Model'] = airbus['Model'].str.replace('A-320', 'A320')

In [34]:
airbus['Model'] = airbus['Model'].apply(lambda x: 'A' + x if not x.startswith('A') else x)

In [35]:
airbus['Aircraft_Category'] = airbus['Aircraft_Category'].fillna('Airplane')

In [36]:
airbus['Engine_Type'] = airbus['Engine_Type'].fillna('Turbo Fan')

In [37]:
airbus['Number_of_Engines'] = airbus['Number_of_Engines'].fillna(2.0)

In [38]:
airbus['Weather_Condition'] = airbus['Weather_Condition'].fillna('UNK')

In [43]:
airbus = airbus.dropna(subset= ['Injury_Severity', 'Aircraft_damage', 'Event_Date'])

In [40]:
airbus['Injury_Severity'] = airbus['Injury_Severity'].apply(lambda x: ''.join(y for y in x if not y.isdigit()))

In [41]:
airbus['Injury_Severity'] = airbus['Injury_Severity'].str.replace('Fatal()', 'Fatal')
airbus['Injury_Severity'].value_counts()

Injury_Severity
Non-Fatal    30
Fatal        13
Incident      7
Name: count, dtype: int64

In [44]:
# Grouping the two datasets and reseting index.
top_manufacturers = pd.concat([boeing, airbus])

In [45]:
top_manufacturers = top_manufacturers.reset_index(drop=True)
top_manufacturers.head()

,Aircraft_Category,Make,Model,Engine_Type,Number_of_Engines,Aircraft_damage,Injury_Severity,Weather_Condition,Event_Date
0,Airplane,Boeing,737,Turbo Fan,2.0,Minor,Incident,VMC,1982-08-05
1,Airplane,Boeing,737,Turbo Fan,2.0,Minor,Incident,IMC,1982-08-12
2,Airplane,Boeing,737,Turbo Jet,2.0,Destroyed,Fatal,VMC,1991-03-03
3,Airplane,Boeing,737,Turbo Fan,2.0,Substantial,Non-Fatal,VMC,1991-10-16
4,Airplane,Boeing,777,Turbo Fan,2.0,Substantial,Non-Fatal,VMC,1997-05-14


In [46]:
# Saving a cleaned dataset as CSV
top_manufacturers.to_csv('cleaned_top_manufacturers_df.csv')

# Exploratory Data Analysis

## Conclusions

## Limitations

## Recommendations

## Next Steps